In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable

In [2]:
use_gpu = torch.cuda.is_available()
use_gpu

True

# Build Model

## Encoder Structure

In [3]:
class LSTMVAE(nn.Module):
    def __init__(self, lstm_layer = 1, seq_len = 10, input_dim = 5, hidden_dim = 10, batch_size = 1):
        super(LSTMVAE, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm_enc = nn.LSTM(input_dim, hidden_dim, lstm_layer)
        self.lstm_dec = nn.LSTM(int(hidden_dim / 2), input_dim, lstm_layer)
        self.fc_mu = nn.Linear(hidden_dim, int(hidden_dim / 2))
        self.fc_logvar = nn.Linear(hidden_dim, int(hidden_dim / 2))
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
        self.seq_len = seq_len
    
    def encode(self, x):
        h1, _ = self.lstm_enc(x)
        h1 = self.tanh(h1)
        return self.fc_mu(h1[-1]), self.fc_logvar(h1[-1])
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            if use_gpu:
                eps = eps.cuda()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def decode(self, z):
        z = z.expand(self.seq_len, -1, -1)
        out, _ = self.lstm_dec(z)
        out = self.tanh(out)
        return out
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        out = self.decode(z)
        return out, mu, z

In [4]:
model = LSTMVAE()
if use_gpu:
    model.cuda()
optimizer = optim.Adam(model.parameters())

In [5]:
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.MSELoss()(recon_x, x.unsqueeze(1))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [9]:
def train(num_epochs=8):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        train_loss = 0
        data = torch.randn([10, 5])
        if use_gpu:
            data = Variable(data.cuda())
        else:
            data = Variable(data)
        model.train()
        optimizer.zero_grad()
        out, mu, z = model(data.unsqueeze(1))
        loss = loss_function(out, data, mu, z)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        print(train_loss)
        print('input: {}'.format(data))
        print('output: {}'.format(out.squeeze()))
    return model

In [10]:
model = train(10)

Epoch 0/9
----------
3.1889357566833496
input: Variable containing:
-1.1771 -0.3941 -0.0791  1.6708  1.1097
 0.2297  0.2026  0.8844  0.3431  0.6123
 0.9808  1.5825 -0.1166 -0.1829 -0.0835
-0.9845  0.1240 -0.1499 -0.4636  0.7095
 1.2399  0.0180 -0.6377 -0.0890 -1.3911
 0.8953 -1.3431 -0.3679  0.7775 -1.0839
-0.0379 -0.8397  1.7879 -1.8201 -2.8483
 0.6125 -0.3064  0.0144  0.2854  0.7347
-0.4322 -0.3648  2.6347 -0.4529  0.1817
-1.9103  0.3955 -1.4532 -0.8341 -1.7567
[torch.cuda.FloatTensor of size 10x5 (GPU 0)]

output: Variable containing:
-0.0339 -0.1853  0.2549 -0.1997 -0.1359
-0.0475 -0.2754  0.3231 -0.2768 -0.2361
-0.0534 -0.3142  0.3337 -0.3051 -0.3018
-0.0561 -0.3315  0.3288 -0.3164 -0.3429
-0.0575 -0.3398  0.3214 -0.3214 -0.3686
-0.0581 -0.3439  0.3151 -0.3238 -0.3847
-0.0585 -0.3461  0.3105 -0.3251 -0.3950
-0.0587 -0.3472  0.3074 -0.3259 -0.4017
-0.0588 -0.3478  0.3054 -0.3264 -0.4060
-0.0588 -0.3481  0.3040 -0.3267 -0.4089
[torch.cuda.FloatTensor of size 10x5 (GPU 0)]

Epoch 1/9